In [ ]:
#### Reading data from online database


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

df1 = pd.read_pickle('covid_demo2') #Covid demographic data  
df2 = pd.read_pickle('demo+cases2') #demographic data+ covid-cases by zipcode

## only numeric data 
df3=pd.DataFrame(df1.drop(['lab_report_date'],axis=1)).astype('int') #Covid demographic data 
df4=pd.DataFrame(df2.drop(['zipcode','Location','County'],axis=1)).astype('int') #demographic data+ covid-cases by zipcode


#filtering/scaling 
new3=df4.iloc[:,[0,1,2,3,4,5,6,9,10,11,16,17,18,19,20,21,22,23,15]]
new3= pd.DataFrame(scaler.fit_transform(new3), columns=new3.columns)
y3=new3.iloc[:,18]
X4=new3.iloc[:,0:18]


## K-means 
from sklearn.cluster import KMeans

## new data basaed on cases 
data_new=df2.iloc[:,17:19].apply(pd.to_numeric)
data_new=pd.DataFrame(scaler.fit_transform(data_new))


### chosing k value 
distortions = []
for i in range(1, 10):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(data_new)
    distortions.append(km.inertia_)

#plotting 
# plt.plot(range(1, 10), distortions, marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('Distortion')
# plt.show()

#### clustering k=3 is the best 

kmeans = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_kmeans = kmeans.fit_predict(data_new)


y_kmeans=y_kmeans.astype('float64')
data1=pd.DataFrame(X4)
data1['cluster']=y_kmeans

##PCA 

from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

pca_model = PCA(n_components=10)
pca=pca_model.fit_transform(X4)
# plt.plot(pca_model.explained_variance_ratio_)
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance')

def myplot(score,coeff,cluster,labels=None):
    
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())

    LABEL_COLOR_MAP = {0 : 'r',1 : 'b',2:'y'}
    label_color = [LABEL_COLOR_MAP[l] for l in cluster]
    
    plt.figure(figsize=(12,12),dpi=80)
    plt.scatter(xs * scalex,ys * scaley,s=40,c=label_color)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'black',alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'green', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
 
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))
    plt.grid()
    plt.rcParams.update({'font.size': 20})
    import matplotlib.patches as mpatches
    classes = ['Low Cases and deaths','Medium Cases and deaths','High Cases and deaths']
    class_colours = ['r','y','b']
    recs = []
    for i in range(0,len(class_colours)):
        recs.append(mpatches.Rectangle((0,0),1,1,fc=class_colours[i]))
    plt.legend(recs,classes,numpoints=1,loc='upper left',fontsize=13)
    plt.savefig("plot2.pdf")
    plt.show()
    
myplot(pca[:,0:2],np.transpose(pca_model.components_[0:2, :]),list(data1.cluster),list(X4.columns))
plt.show()
